In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.backends.cudnn as cudnn
from tensorboardX import SummaryWriter
import model.simplenet as simplenet
import matplotlib.pyplot as plt
import torch.utils.data as Data
import imageio
import numpy as np
import random
import pandas as pd

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False
    #torch.backends.cudnn.benchmark = True #for accelerating the running
    return

def savefig(name):
    plt.savefig(name,dpi=600, bbox_inches='tight')
    return

def change_weight(weight):
    '''
    Change the weight matrix in quadratic neural network.
    '''
    model_dict = model.state_dict()
    model_dict['classifier.0.weight_a'] = weight
    model.load_state_dict(model_dict)
    
    return

from sklearn.datasets import load_wine


wine = load_wine()['data']
for i in range(13):
    wine[:,i] = wine[:,i]/max(wine[:,i])

target = torch.from_numpy(load_wine()['target'])

device = torch.device('cpu')
writer = SummaryWriter()

train_dataset = torch.from_numpy(wine)
valid_dataset = torch.from_numpy(wine)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=178, shuffle=False)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=178)

EI_distribution = torch.bernoulli(torch.ones(13)*0.75)
print('number of excitatory neuron:{}'.format(EI_distribution.sum()))
kappa_matrix = -torch.ones(13, 13)
kappa_matrix[EI_distribution==0,:] = 1
kappa_matrix[:,EI_distribution==0] = 1

model = simplenet.SimpleNet_0(num_eigens=5, EI_distribution=EI_distribution)

print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
test_accuracy = []
train_accuracy = [0]
all_train_loss = []
image_list = []
best_prec = 0
min_loss = 1

for epoch in range(0, 10000):

    for param_group in optimizer.param_groups:
        param_group['lr'] = 0.01

    model.train()
    # train for one epoch
    for i, input in enumerate(train_loader):
        train_total = 0
        train_correct = 0
        train_loss = 0
                    
        
        # compute output
        output = model(input.to(torch.float32))
        loss = criterion(output, target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward() 
        optimizer.step()

        weight_a = model.state_dict()['classifier.0.weight_a']
        new_weight = weight_a.data
        for j in range(3):
            new_weight[j,(new_weight[j,:,:]*kappa_matrix)<0] = 0

        change_weight(new_weight)  

        _, predicted = torch.max(output.data, 1)
        train_total = target.size(0)
        train_correct = (predicted == target).sum().item()
        prec = train_correct / train_total

        if (i) % 300 == 0:
            train_accuracy.append(prec)
            all_train_loss.append(loss)

            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.5f}, Train_Acc:{:.2f}%'.format(epoch+1, 10000, i, len(train_loader), loss, prec*100))

    
            model.eval()
            valid_correct = 0
            valid_total = 0
            with torch.no_grad():
                total_loss = 0
                for j, input in enumerate(valid_loader):
                    output = model(input.to(torch.float32))
                    
                    _, predicted = torch.max(output.data, 1)
                    valid_total = output.shape[0]
                    valid_correct = (predicted == target).sum().item()
                    loss = criterion(output, target)
                
                    prec = valid_correct / valid_total
                    print('Accuary on test images:{:.2f}%, loss:{:.2f}'.format(prec*100, loss))
                    test_accuracy.append(prec)
                    best_prec = max(prec, best_prec)
                    min_loss = min(loss,min_loss)
        
print('Best accuracy is: {:.2f}%, Minimum loss is: {:.4f}'.format(best_prec*100, min_loss))

number of excitatory neuron:8.0
SimpleNet_0(
  (classifier): Sequential(
    (0): Dales_General_quadratic(in_features=13, out_features=3, bias=False)
  )
)
Epoch [1/10000], Step [0/1], Loss: 1.10338, Train_Acc:33.15%
Accuary on test images:33.15%, loss:1.09
Epoch [2/10000], Step [0/1], Loss: 1.09305, Train_Acc:33.15%
Accuary on test images:33.15%, loss:1.08
Epoch [3/10000], Step [0/1], Loss: 1.08344, Train_Acc:33.15%
Accuary on test images:33.15%, loss:1.07
Epoch [4/10000], Step [0/1], Loss: 1.07456, Train_Acc:33.15%
Accuary on test images:33.15%, loss:1.07
Epoch [5/10000], Step [0/1], Loss: 1.06630, Train_Acc:33.15%
Accuary on test images:33.15%, loss:1.06
Epoch [6/10000], Step [0/1], Loss: 1.05860, Train_Acc:33.15%
Accuary on test images:34.83%, loss:1.05
Epoch [7/10000], Step [0/1], Loss: 1.05141, Train_Acc:34.83%
Accuary on test images:34.83%, loss:1.04
Epoch [8/10000], Step [0/1], Loss: 1.04465, Train_Acc:34.83%
Accuary on test images:34.83%, loss:1.04
Epoch [9/10000], Step [0/1],